In [21]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('vaccine_preprocessed_2.csv',sep='|')

In [22]:
df.head(1)

,item_id,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
0,1,AZ,澎湖確診數升 兒童疫苗接種率已逾7成,澎湖COVID-19（2019冠狀病毒疾病）確診數攀升，澎湖縣政府今天公布新增182例，還有...,0.0,"['澎湖5至11歲兒童疫苗接種種率已逾7成', '澎湖縣政府今天公布新增182例確診案例',...","[('澎湖', 5), ('疫苗', 5), ('接種', 4), ('確診', 3), (...","['澎湖', 'COVID-19', '（', '2019', '冠狀', '病毒', '疾...","['澎湖', '冠狀', '病毒', '疾病', '確診數', '澎湖縣', '政府', '...","[NerToken(word='澎湖', ner='GPE', idx=(0, 2)), N...","[('澎湖', 'Nc'), ('COVID-19', 'FW'), ('（', 'PARE...",https://news.google.com/articles/CBMiMmh0dHBzO...


In [23]:
df.shape

(796, 12)

In [24]:
def filter_df_via_tokens(query_keywords, cond, cate):
    
    # proceed filtering
    if (cate == "全部"):
        condition = (df.categories == 'AZ') | (df.categories == 'BNT') | (df.categories == '莫德納') | (df.categories == '高端')# "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = df.categories == cate

    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.tokens_v2.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.tokens_v2.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [25]:
def filter_df_via_content(query_keywords, cond, cate):
    
    # proceed filtering
    if (cate == "全部"):
        condition = (df.categories == 'AZ') | (df.categories == 'BNT') | (df.categories == '莫德納') | (df.categories == '高端')# "全部"類別不必過濾新聞種類
    else:
        # category新聞類別條件
        condition = df.categories == cate

    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.contents.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.contents.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query

In [26]:
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0}
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.75:
            sentiCount['pos'] += 1
        elif senti <= 0.4:
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1
    for polar in sentiCount :
        sentiPercnt[polar]=int(sentiCount[polar]/numberOfArticle*100)
        #sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle,2)
    return sentiCount, sentiPercnt


In [27]:
query_keywords = ['陳時中']
cond='or'
cate='全部'
df_query = filter_df_via_tokens(query_keywords, cond, cate)
sentiCount, sentiPercnt = get_article_sentiment(df_query)

In [28]:
sentiCount

{'pos': 1, 'neg': 219, 'obj': 3}

In [29]:
sentiPercnt

{'pos': 0, 'neg': 98, 'obj': 1}

In [10]:
# sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
#                                            str(sentiPercnt['neg']))
[sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

[0, 1, 98]

In [31]:
# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = ['AZ', 'BNT', '莫德納', '高端', '全部']


def count_keyword(df_query, query_keywords):

    cate_occurrence = {}
    cate_freq = {}
    
    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0   # {'政治':0, '科技':0}
        cate_freq[cate] = 0

    for idx, row in df_query.iterrows():
        # count the number of articles各類別篇數統計
        cate_occurrence[row.categories] += 1  #   {'政治':+1, '科技':0}
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency各類別次數統計
        tokens = eval(row.tokens_v2)
        # 計算這一篇文章的tokens中重複含有多少個這些關鍵字(頻率)
        freq = len([word for word in tokens if (word in query_keywords)]) 
        cate_freq[row.categories] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']


    
    return cate_freq, cate_occurrence, total_articles, total_frequency

In [32]:
query_keywords = ['陳時中']
cond='or'
cate='全部'
df_query = filter_df_via_content(query_keywords, cond, cate)
count_keyword(df_query, query_keywords)

({'AZ': 165, 'BNT': 177, '莫德納': 191, '高端': 197, '全部': 730},
 {'AZ': 60, 'BNT': 56, '莫德納': 57, '高端': 57, '全部': 230},
 230,
 730)

In [33]:
df_query.head(1)

,item_id,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
4,5,AZ,AZ疫苗第2劑本週開放50至64歲預約 可望雙十連假前開打,指揮中心今天公布，AZ疫苗第2劑改由線上預約，即起開放9月30日前間隔滿10週以上50歲至6...,0.0,['中央流行疫情指揮中心原規劃COVID-19（2019冠狀病毒疾病）疫苗第10輪10月3日...,"[('疫苗', 12), ('預約', 8), ('接種', 7), ('意願', 6), ...","['指揮', '中心', '今天', '公布', '，', 'AZ', '疫苗', '第2'...","['指揮', '中心', '疫苗', '線上', '預約', '開放', '間隔', '民眾...","[NerToken(word='今天', ner='DATE', idx=(4, 6)), ...","[('指揮', 'Na'), ('中心', 'Nc'), ('今天', 'Nd'), ('公...",https://news.google.com/articles/CBMiN2h0dHBzO...


In [37]:
# 定義PK對象
# 通常會多個關鍵字代表該PK對象，['柯文哲','柯p','柯P']
list_pkNames = ['AZ', 'BNT', '莫德納', '高端']
list_pkKeywordSet = [['AZ'], ['BNT'], ['莫德納'], ['高端']]


# 線條顏色
list_colors = ["rgba(0,128,255,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)', 'rgba(0,128,0,0.2)'] 
#list_colors = ['green', 'red', 'blue']

# 人頭圖案
list_photos = [
    "https://doqvf81n9htmm.cloudfront.net/data/changchi_177/%EF%BC%98%EF%BC%98%EF%BC%98/654654654654654654/shutterstock_1866527854.jpg",
    "https://doqvf81n9htmm.cloudfront.net/data/changchi_177/%EF%BC%98%EF%BC%98%EF%BC%98/654654654654654654/shutterstock_1866787507.jpg",
    "https://storage.googleapis.com/www-cw-com-tw/article/202202/article-620cda8fb4003.jpg",
    "https://upload.wikimedia.org/wikipedia/zh/thumb/b/b3/Flag_of_Democratic_Progressive_Party.svg/1200px-Flag_of_Democratic_Progressive_Party.svg.png"
]

# 準備長條圖 線圖 總篇數 總次數等數據
list_freq_daily_line_chart = []
list_freq_news_category = []
list_sentimentInfo = []

list_total_articles=[]
list_total_frequency=[]

cate = "全部"
cond = 'or'

selectedCategories = ['全部', 'AZ', 'BNT', '莫德納', '高端']
# selectedCategories = ['全部','政治', '產經', '生活', '國際', '社會', '兩岸']

for query_keywords in list_pkKeywordSet:

    # Filter news   
    df_query = filter_df_via_content(query_keywords, cond, cate)
    #df_query = filter_df_via_tokens(df, query_keywords, cond, cate, weeks)
    
    # Get bar chart data, total articals and frequecy 
    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
    list_total_articles.append(total_articles)
    list_total_frequency.append(total_frequency)


    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    cate_freq = [cate_occurrence[k] for k in selectedCategories]
    list_freq_news_category.append(cate_freq)

    # Get sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    #sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    senti = [sentiPercnt[p] for p in ['pos','obj','neg']] # 只要取用pos,obj, neg的情緒數字篇數百分比
    list_sentimentInfo.append(senti)

# We need all the following data to display on our frontend page 有一大堆數據要送到前端去展示啊
pk_data =  {
           'list_pkNames': list_pkNames,
           'list_colors': list_colors,
           'list_photos': list_photos,
           'list_freq_news_category': list_freq_news_category,
           'list_category': selectedCategories,
           'list_sentiInfo': list_sentimentInfo,
           'list_total_articles': list_total_articles,
           'list_total_frequency': list_total_frequency
           }

In [38]:
pk_data

{'list_pkNames': ['AZ', 'BNT', '莫德納', '高端'],
 'list_colors': ['rgba(0,128,255,0.2)',
  'rgba(0,255,255,0.2)',
  'rgba(0,0,255,0.2)',
  'rgba(0,128,0,0.2)'],
 'list_photos': ['https://doqvf81n9htmm.cloudfront.net/data/changchi_177/%EF%BC%98%EF%BC%98%EF%BC%98/654654654654654654/shutterstock_1866527854.jpg',
  'https://doqvf81n9htmm.cloudfront.net/data/changchi_177/%EF%BC%98%EF%BC%98%EF%BC%98/654654654654654654/shutterstock_1866787507.jpg',
  'https://storage.googleapis.com/www-cw-com-tw/article/202202/article-620cda8fb4003.jpg',
  'https://upload.wikimedia.org/wikipedia/zh/thumb/b/b3/Flag_of_Democratic_Progressive_Party.svg/1200px-Flag_of_Democratic_Progressive_Party.svg.png'],
 'list_freq_news_category': [[335, 196, 33, 47, 59],
  [471, 88, 193, 103, 87],
  [528, 99, 100, 194, 135],
  [244, 45, 37, 45, 117]],
 'list_category': ['全部', 'AZ', 'BNT', '莫德納', '高端'],
 'list_sentiInfo': [[4, 0, 95], [4, 0, 95], [4, 0, 95], [4, 0, 95]],
 'list_total_articles': [335, 471, 528, 244],
 'list_total_

In [39]:
df_data_pk = pd.DataFrame(list(pk_data.items()),columns=['name','value'])

In [40]:
df_data_pk

,name,value
0,list_pkNames,"[AZ, BNT, 莫德納, 高端]"
1,list_colors,"[rgba(0,128,255,0.2), rgba(0,255,255,0.2), rgb..."
2,list_photos,[https://doqvf81n9htmm.cloudfront.net/data/cha...
3,list_freq_news_category,"[[335, 196, 33, 47, 59], [471, 88, 193, 103, 8..."
4,list_category,"[全部, AZ, BNT, 莫德納, 高端]"
5,list_sentiInfo,"[[4, 0, 95], [4, 0, 95], [4, 0, 95], [4, 0, 95]]"
6,list_total_articles,"[335, 471, 528, 244]"
7,list_total_frequency,"[13, 1, 2194, 73]"


In [41]:
## 存成csv格式檔案
df_data_pk.to_csv('pk_vaccine_election2.csv',sep=',', index=None)

## Django function

In [ ]:
from django.http import JsonResponse
import pandas as pd
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt

def load_data_pk():
    # Read data from csv file
    df_data_pk = pd.read_csv('app_taipei_mayor/dataset/pk_taipei_mayor_election.csv')
    
    global data
    data={}
    for k,v in zip(df_data_pk.name, df_data_pk.value):
        data[k]=eval(v)
    
    # 沒用到的變數刪除之
    del df_data_pk

# load pk data
load_data_pk()

def home(request):
    return render(request,'app_taipei_mayor/home.html')

# csrf_exempt is used for POST
# 單獨指定這一支程式忽略csrf驗證
@csrf_exempt
def api_get_taipei_mayor_data(request):
    return JsonResponse(data)

print('app taipei mayor leaderboard was loaded!')
